# Plotly pour des représentations interactives

### Utilisation de Plotly

Plotly est un autre package intéressant pour la création de graphiques. Il est basé sur javascript. Son principal intérêt réside dans la création de graphiques interactifs. Il est édité par la société canadienne *Plotly* et est également intégré dans le package **dash** de création de dashboards.

Nous allons principalement utiliser le module **plotly.express** dont l'API est particulièrement simple. Il existe un module plus complet que nous n'aborderons pas : **plotly.graph_objects**.

Les graphiques sont produits avec une barre d'outils offrant différentes fonctions (snapshot, zoom, sélection, recentrage...). Il est possible de configurer ou de cacher ce menu avec l'instrcution : <code>fig.show(config={'displayModeBar': False})</code>.

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px

**Exemples inspirés du site de plotly**

#### bar

In [ ]:
# bar
df = px.data.gapminder().query("country == 'France'")
px.bar(df, x='year', y='pop')

#### pie

In [ ]:
# pie
df = px.data.gapminder().query("year == 2007").query("continent == 'Europe'")
df.loc[df['pop'] < 5.0e6, 'country'] = 'Autres' # Represent only large countries
px.pie(df, values='pop', names='country', title='Population du continent européen')

#### strip

In [ ]:
# strip
df = px.data.tips()
px.strip(df, x="total_bill", y="day")

#### scatter

In [ ]:
# scatter
px.scatter(df, x="total_bill", y="tip", color="sex")

#### box

In [ ]:
# box
px.box(df, y="total_bill")

#### violin

In [ ]:
# violin
px.violin(df, y="total_bill", color="sex")

#### histogram

In [ ]:
# histogram
px.histogram(df, x="total_bill", nbins=10)

#### scatter_matrix

In [ ]:
# scatter_matrix
px.scatter_matrix(df)

#### density_heatmap

In [ ]:
# density_heatmap
px.density_heatmap(df, x="total_bill", y="tip", marginal_x="histogram", marginal_y="histogram")

#### facet

In [ ]:
# grid + histogram
px.histogram(df, x="total_bill", facet_row="day", facet_col="sex")

In [ ]:
# grid + scatter
px.scatter(df, x="total_bill", y="tip", facet_row="day", facet_col="sex")

### treemap et sunburst

Une treemap (resp. sunburst) ou carte proportionnelle est une représentation rectangulaire (resp. circulaire) de données hiérarchiques dans un espace limité.

In [ ]:
# dataset
fortune = pd.read_csv("./Fortune_1000.csv", na_values="-")
fortune = fortune.fillna(0)
fortune.head()

In [ ]:
# treemap
px.treemap(fortune,
          path=["sector", "company"],
          values="Market Cap")

In [ ]:
# on limite aux 4 première capitalisations par secteur
fortune2 = fortune.groupby("sector").apply(lambda g: g.nlargest(4, "Market Cap"))

# sunburst
px.sunburst(fortune2,
          path=["sector", "company"],
          values="Market Cap")

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise</h3>
    <ul>
        <li>Produire une treemap avec la hiérarchie à 3 niveaux : état ("state"), ville ("city"), société ("company") avec le nombre d'employés ("num. of employees").</li>
    </ul>
    <h4>Facultatif</h4>
    <ul>
        <li>A partir du fichier "correspondance-code-insee-code-postal.csv", produire une treemap avec la hiérarchie à 3 niveaux : "Région", "Département", "Commune" avec la "Population" limitée aux 3 villes les plus peuplées de chaque département</li>
    </ul>
</div>

In [ ]:
# %load exos_viz/exo_11.py

### Cartes choroplèthes

Une carte choroplèthe est une carte thématique où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique.

Pour produire une carte choroplèthe, il faut récupérer un fichier *geojson* de la partie du monde considérée. Ce fichier contient entre autres 2 informations importantes :
- Une clé qui désigne chaque sous-région,
- La description d'un polygone ou d'un multi-polygone sous la forme d'une liste de coordonnées (latitude et longitude),

Il faut ensuite faire coïncider les valaeurs de la clé du fichier *geojson* avec celles de la colonne du DataFrame qui contient les données relatives à chaque sous-région.

Il est possible ensuite d'utiliser différents fonds de cartes.

Le fichier "./data/departements.geojson" utilisé ci-après provient du site : https://france-geojson.gregoiredavid.fr/ et il faut analyser le fichier pour trouver la clé correspondant à chaque sous-région.

In [ ]:
import json

with open("./departements.geojson") as f:
    departements = json.loads(f.read())

departements

Analyse du fichier geojson :
- departements["features"] est une liste de 96 dictionnaires (les différents départements avec '2A' et '2B')
- chaque dictionnaire contient les clés :
    - 'type': 'Feature'
    - 'geometry': informations géométriques (polygones...)
    - 'properties' : informations d'identification qui permettent de référencer chaque dictionnaire

In [ ]:
# analyse du dictionnaire "departements"
departements["features"][0]["properties"]

In [ ]:
# liste des valeurs de la clé "code"
codes = [f["properties"]["code"] for f in departements["features"]]
codes.sort()
print(*codes)

In [ ]:
# dataset 
geo = pd.read_csv("./correspondance-code-insee-code-postal.csv",
                 sep=";",
                 )
geo.head()

On voit que l'on va pouvoir utiliser la colonne "Code Département" pour référencer les dictionnaires du fichier *geojson*. La clé d'accès sera : "properties.code".

In [ ]:
# liste des "Code Département"
codes2 = geo["Code Département"].unique()
codes2.sort()
print(*codes2)

On calcule le nombre d'habitants par département.

In [ ]:
# population des départements
df = geo.groupby("Code Département", as_index=False)["Population"].sum()
df.head()

On produit la carte directement en donnant notamment les 4 informations :
- le dictionnaire issu du fichier *geojson* : `geojson=departements`,
- la clé dans le DataFrame : `locations='Code Département'`
- la clé dans le fichier *geojson* : `featureidkey='properties.code'`
- la grandeur représentée : `color='Population'`

In [ ]:
# choropleth_mapbox
px.choropleth_mapbox(data_frame=df,
                     geojson=departements,
                     locations='Code Département',
                     color='Population',
                     featureidkey='properties.code',
                     color_continuous_scale="teal",
                     mapbox_style="carto-positron",
                     zoom=4.0,
                     center = {"lat": 47.0, "lon": 0.0},
                     opacity=0.5,
                     labels={'Population': 'Population en milliers'}
                    )

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 12</h3>
    <ul>
        <li>Produire la même carte choroplèthe en utilisant non pas le n° mais le nom du département. Que constate-t-on ?</li>
    </ul>
</div>

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 13</h3>
    <ul>
        <li>Produire une carte choroplèthe avec la moyenne de l'Altitude Moyenne de chaque département, en utilisant le fond de carte "stamen-terrain" et le nuancier "reds".</li>
    </ul>
</div>

<div class="alert alert-info">
    <h3><i class="fa fa-info-circle"></i> Documentation</h3>
    <p><a href="https://plotly.com/python/plotly-express/">Plotly Express</a></p>
    <p><a href="https://plotly.com/python/builtin-colorscales/">Built-in Continuous Color Scales</a></p>
    <p><a href="https://plotly.com/python/mapbox-layers/">Mapbox Map Layers</a></p>
</div>